In [1]:
import os
import transformers
import math
from tqdm import tqdm
from transformers import (
    GPT2Tokenizer,
    GPT2LMHeadModel,
    T5Tokenizer
)

In [21]:
root_dir = "/projects/ogma2/users/andrewsi/control-data2text"

def compute_perplexity(preds):
    e2e_lm_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    e2e_lm = GPT2LMHeadModel.from_pretrained(f"{root_dir}/transformers/examples/language-modeling/exp/e2e_targets/gpt2-01/checkpoint-7458")
    e2e_lm.to("cuda:3")
    ppls = []
    for pred in tqdm(preds):
        inputs = e2e_lm_tokenizer(pred, return_tensors='pt').to("cuda:3")
        outputs = e2e_lm(**inputs, labels=inputs['input_ids'])
        ppls.append(math.exp(outputs.loss))
    return round((sum(ppls) / len(ppls)), 4)

In [11]:
preds_file = "/projects/ogma2/users/andrewsi/control-data2text/transformers/examples/seq2seq/exp/e2e/e2e_k_5_t5_small/mask_y_ref.valid.txt"
with open(preds_file, "r") as f:
    preds = [pred.replace("_", " ").strip() for pred in f]

In [13]:
preds[:10]

['<UNK> <UNK> near <UNK> has a <UNK> star rating . Prices start at £30 .',
 '<UNK> is located near <UNK> in the <UNK> . <UNK> prices are <UNK> and <UNK> are <UNK> .',
 '<UNK> is a <UNK> providing dine in meals in the <UNK> price range . It is located in at <UNK> .',
 '<UNK> is a <UNK> that offers <UNK> food though with a poor rating',
 '<UNK> is a <UNK> <UNK> located in the <UNK> , near <UNK> .',
 '<UNK> <UNK> is not a family <UNK> . It is located near <UNK> .',
 '<UNK> serves <UNK> food costing more that £30 and has a <UNK> customer rating . It is located in the <UNK> and has a <UNK> atmosphere .',
 '<UNK> is in <UNK> near <UNK> and serves <UNK> food with a price range is <UNK> .',
 '<UNK> is a <UNK> that cooks <UNK> food it is located by the <UNK> and the price range is <UNK>',
 '<UNK> is a <UNK> <UNK> with <UNK> customer ratings located near <UNK> in the <UNK> area .']

In [22]:
compute_perplexity(preds)

100%|██████████| 6300/6300 [01:58<00:00, 53.33it/s]


3113.2197

In [2]:
tokenizer = T5Tokenizer.from_pretrained("t5-small")

In [6]:
text = "The_Waterman is a not_family_friendly English restaurant near Crowne_P"
ids = tokenizer(text)["input_ids"]
print(ids)
len(ids)

[37, 834, 28632, 348, 19, 3, 9, 59, 834, 15474, 834, 4905, 1566, 2062, 1084, 15633, 15, 834, 345, 1]


20